In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

In [4]:
groq_api_key=os.getenv("GROQ_API_KEY")

In [5]:
llm=ChatGroq(groq_api_key=groq_api_key,model="gemma:2b")

In [11]:
import bs4

from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# does not work for me instead using ollama

In [20]:
embeddings=OllamaEmbeddings()
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [43]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import bs4

# Initialize loader with broader parsing
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",),
    requests_kwargs={"headers": {"User-Agent": "Mozilla/5.0"}},
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name=["article", "h1", "h2", "div", "section"]  # Broader tags
        )
    )
)

# Load documents
try:
    docs = loader.load()
except Exception as e:
    print(f"Error loading page: {e}")
    docs = []

# Check content
if not docs:
    print("No documents loaded. Check URL or network.")
elif not docs[0].page_content:
    print("No content found. Try adjusting SoupStrainer or removing it.")
else:
    print(f"Loaded {len(docs)} document(s). Content preview: {docs[0].page_content[:200]}")

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # Create vector store
    if splits and splits[0].page_content:
        vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
        retriever = vectorstore.as_retriever()
        print("Retriever created:", retriever)
    else:
        print("No content in splits to process.")

Loaded 1 document(s). Content preview: 
Lil'Log

















|





      Reward Hacking in Reinforcement Learning
    
Date: November 28, 2024  |  Estimated Reading Time: 37 min  |  Author: Lilian Weng


 


Table of Contents



Backg


ValueError: Error raised by inference API HTTP code: 404, {"error":"model \"llama2\" not found, try pulling it first"}

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Check if docs has content before creating vectorstore
if splits and splits[0].page_content:
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    retriever = vectorstore.as_retriever()
    retriever
else:
    print("No content found in documents to process")

No content found in documents to process


In [32]:
## prompt template

system_prompt=(
    ""
    "/n/n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [34]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
ragchain=create_retrieval_chain(retriever,question_answer_chain)


NameError: name 'retriever' is not defined

In [35]:
response=ragchain.invoke({"input":"What is self reflection"})

repsonse["answer"]


NameError: name 'ragchain' is not defined

In [36]:
ragchain.invoke({"input":"how do we implement it"})
response["answer"]

NameError: name 'ragchain' is not defined

In [37]:
# making chat history

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt=(
    ""
    ""
    ""
)

contextualize_q_system_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
 
 
    ]
)

In [39]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_system_prompt)
history_aware_retriever

NameError: name 'retriever' is not defined

In [44]:
question_answer_chain=create_stuff_documents_chain(llm,history_aware_retriever)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

NameError: name 'history_aware_retriever' is not defined

In [46]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]

question="What is Slef Relfcetion"
repsonse=rag_chain.invoke({'input':question,"chat_history":chat_history})

NameError: name 'rag_chain' is not defined

In [47]:
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

chat_history


NameError: name 'response1' is not defined